In [13]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from pyimagesearch.livenessnet import LivenessNet
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.utils import np_utils
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import os

In [30]:
# initialize the initial learning rate, batch size, and number of
# epochs to train for
INIT_LR = 1e-4
BS = 8
EPOCHS = 100

In [4]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
imagePaths = list(paths.list_images("dataset"))
data = []
labels = []

for imagePath in imagePaths:
	# extract the class label from the filename, load the image and
	# resize it to be a fixed 32x32 pixels, ignoring aspect ratio
	label = imagePath.split(os.path.sep)[-2]
	image = cv2.imread(imagePath)
	try:
		image = cv2.resize(image, (32, 32))
	except:
		print("Imagem com erro",imagePath)

	# update the data and labels lists, respectively
	data.append(image)
	labels.append(label)



[INFO] loading images...


In [5]:
# convert the data into a NumPy array, then preprocess it by scaling
# all pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0

# encode the labels (which are currently strings) as integers and then
# one-hot encode them
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = np_utils.to_categorical(labels, 2)

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.25, random_state=42)

# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
	width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
	horizontal_flip=True, fill_mode="nearest")

In [31]:
# initialize the optimizer and model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model = LivenessNet.build(width=32, height=32, depth=3,
	classes=len(le.classes_))
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] compiling model...


In [32]:
# train the network
print("[INFO] training network for {} epochs...".format(EPOCHS))
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS)

[INFO] training network for 100 epochs...
Epoch 1/100
1953/1953 [==============================] - 34s 17ms/step - loss: 0.7396 - accuracy: 0.6601 - val_loss: 0.2320 - val_accuracy: 0.9361
Epoch 2/100
1953/1953 [==============================] - 35s 18ms/step - loss: 0.4554 - accuracy: 0.7969 - val_loss: 0.1183 - val_accuracy: 0.9704
Epoch 3/100
1953/1953 [==============================] - 35s 18ms/step - loss: 0.3672 - accuracy: 0.8423 - val_loss: 0.0987 - val_accuracy: 0.9750
Epoch 4/100
1953/1953 [==============================] - 37s 19ms/step - loss: 0.3205 - accuracy: 0.8663 - val_loss: 0.0634 - val_accuracy: 0.9845
Epoch 5/100
1953/1953 [==============================] - 35s 18ms/step - loss: 0.2901 - accuracy: 0.8833 - val_loss: 0.0540 - val_accuracy: 0.9893
Epoch 6/100
1953/1953 [==============================] - 35s 18ms/step - loss: 0.2571 - accuracy: 0.8969 - val_loss: 0.0446 - val_accuracy: 0.9889
Epoch 7/100
1953/1953 [==============================] - 34s 17ms/step - los

1953/1953 [==============================] - 35s 18ms/step - loss: 0.0958 - accuracy: 0.9693 - val_loss: 0.0075 - val_accuracy: 0.9985
Epoch 57/100
1953/1953 [==============================] - 35s 18ms/step - loss: 0.0926 - accuracy: 0.9695 - val_loss: 0.0106 - val_accuracy: 0.9967
Epoch 58/100
1953/1953 [==============================] - 36s 19ms/step - loss: 0.0835 - accuracy: 0.9727 - val_loss: 0.0146 - val_accuracy: 0.9958
Epoch 59/100
1953/1953 [==============================] - 37s 19ms/step - loss: 0.0997 - accuracy: 0.9670 - val_loss: 0.0057 - val_accuracy: 0.9992
Epoch 60/100
1953/1953 [==============================] - 36s 18ms/step - loss: 0.0903 - accuracy: 0.9727 - val_loss: 0.0072 - val_accuracy: 0.9967
Epoch 61/100
1953/1953 [==============================] - 37s 19ms/step - loss: 0.0853 - accuracy: 0.9714 - val_loss: 0.0039 - val_accuracy: 0.9988
Epoch 62/100
1953/1953 [==============================] - 36s 18ms/step - loss: 0.0874 - accuracy: 0.9710 - val_loss: 0.0097 

In [33]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=BS)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=le.classes_))

[INFO] evaluating network...
              precision    recall  f1-score   support

        fake       1.00      1.00      1.00      2424
        real       1.00      1.00      1.00      2786

    accuracy                           1.00      5210
   macro avg       1.00      1.00      1.00      5210
weighted avg       1.00      1.00      1.00      5210



In [34]:
print("[INFO] Plotting stats")
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, EPOCHS), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, EPOCHS), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, EPOCHS), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, EPOCHS), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot_100.png")

[INFO] Plotting stats


In [35]:
# save the network to disk
print("[INFO] serializing network to '{}'...".format("model"))
model.save("model_100")

[INFO] serializing network to 'model'...
INFO:tensorflow:Assets written to: model_100/assets
